## Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

Food|Cost|Availability
----|----|------------
Bread|0.1|4
Milk|0.5|3
Eggs|0.12|1
Meat|0.9|2
Cake|1.3|2

A meal must contain at least the following amount of each nutrient

Nutrient|Minimum quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

 <h3 align="center"> Diet problem formulation</h3> 
 
- Sets
    - $I$: food types
    - $J$: nutrients
    
- Parameters
    - $c_i$: unit cost of food type $i \in I$ 
    - $q_i$: available quantity of food type $i \in I$
    - $b_j$: minimum quantity of nutrient $j \in J$ required
    - $a_{ij}$: quantity of nutrient $j \in J$ per unit of food of type $i \in I$
    
- Variables
    - $x_i$: quantity of food of type $i \in I$ included in the diet
    
- Model
    
\begin{array}{lll}
  \min & \sum_{i\in I} c_{i} x_{i} \qquad & & \text{(cost)}\\
  \textrm{s.t.} & \sum_{i\in I}a_{ij}x_{ij} \ge b_j & j \in J & \text{(min nutrients)}\\
  & x_i \leq q_i & i \in I & \text{(availability)}\\
  & x_i \geq 0 &  i \in I & \text{(nonnegativity)}
\end{array}


In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# We need to import the package mip
import mip

In [ ]:
# Food
I = {'Bread', 'Milk', 'Eggs', 'Meat', 'Cake'}

# Nutrients
J = {'Calories', 'Proteins', 'Calcium'}

# Cost in Euro per hg of food
c = {'Bread':0.1, 'Milk':0.5, 'Eggs':0.12, 'Meat':0.9, 'Cake':1.3}

# Availability per hg of food
q = {'Bread':4, 'Milk':3, 'Eggs':1, 'Meat':2, 'Cake':2}

# Minimum nutrients 
b = {'Calories':600, 'Proteins':50, 'Calcium':0.7}

# Nutrients per hf of food
a = {('Bread', 'Calories'):30, ('Milk', 'Calories'):50, ('Eggs', 'Calories'):150, ('Meat', 'Calories'):180, ('Cake', 'Calories'):400,
    ('Bread', 'Proteins'):5, ('Milk', 'Proteins'):15, ('Eggs', 'Proteins'):30, ('Meat', 'Proteins'):90, ('Cake', 'Proteins'):70,
    ('Bread', 'Calcium'):0.02, ('Milk', 'Calcium'):0.15, ('Eggs', 'Calcium'):0.05, ('Meat', 'Calcium'):0.08, ('Cake', 'Calcium'):0.01}

Now we create an empty model and add the variables

In [ ]:
# Define a model
model = mip.Model()

# Define variables
x = [model.add_var(name=i, lb=0) for i in I]

In [ ]:
# Define the objective function
model.objective = mip.minimize(mip.xsum(x[i]*c[food] for i, food in enumerate(I)))

In [ ]:
# CONSTRAINTS
# Availability constraint
for i, food in enumerate(I):
    model.add_constr(x[i]<= q[food])

# Minimum nutrients
for j in J:
    model.add_constr(mip.xsum(x[i]*a[food, j] for i, food in enumerate(I))>= b[j])

The model is complete. Let us solve it and print the optimal solution.

In [ ]:
# Optimizing command
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [ ]:
# Optimal objective function value
model.objective.x

3.37

In [ ]:
# Printing the variables values
for i in model.vars:
    print(i.name, i.x)

Eggs 1.0
Milk 3.0
Cake 0.0
Bread 3.9999999999999996
Meat 1.5000000000000002


## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


 <h3 align="center">Oil blending formulation</h3> 
 
- Sets
    - $I$: set of oil types
    - $J$: set of gasoline types
    
- Parameters
    - $c_i$: unit cost for oil of type $i \in I$ 
    - $b_i$: availability of oil of type $i \in I$
    - $r_j$: price of gasoline of type $i \in I$
    - $b_i$: minimum quantity of nutrient $i \in I $ required
    - $q_{ij}^{\max}$: maximum quantity (percentage) of oil of type $i \in I$ for gasoline of type $j \in J$
    - $q_{ij}^{\min}$: minimum quantity (percentage) of oil of type $i \in I$ for gasoline of type $j \in J$
    
- Variables
    - $x_{ij}$: units of oil of type $i \in I$ used for gasoline of type $j \in J$
    - $y_j$: units of gasoline of type $j \in J$ that are produced
    
- Model
    
\begin{array}{lll}
  \max & \sum_{j \in J}r_j y_j-\sum_{i \in I,j\in J} c_{j} x_{ij} \qquad & & \text{(revenue)}\\
  \textrm{s. t.} & \sum_{j \in J}x_{ij} \leq b_i & i \in I & \text{(availability)}\\
  & \sum_{i \in I}x_{ij}  = y_j & j \in J & \text{(conservation)}\\
  & x_{ij} \leq q_{ij}^{\max}y_j & i \in I, j \in J & \text{(maximum qty)}\\
  & x_{ij} \geq q_{ij}^{\min}y_j & i \in I, j \in J & \text{(minimum qty)}\\
  & x_{ij},y_j \geq 0 &  i \in I,j \in J & \text{(nonnegativity)}
\end{array}


In [ ]:
# Set of oil types
I = range(4)

# Set of gasoline types
J = {'A', 'B', 'C'}

# Unit cost for oil of type i 
c = {0:9, 1:7, 2:12, 3:6}

# Availability of oil type i
b = {0:5000, 1:2400, 2:4000, 3:1500}

# Price of gasoline of type j
r = {'A':12, 'B':18, 'C':10}

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
    for j in J:
        q_max[(str(i), j)] = 1
q_max[('2', 'A')] = 0.3
q_max[('1', 'C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
    for j in J:
        q_min[(str(i), j)] = 0
q_min[('1', 'A')] = 0.2
q_min[('2', 'B')] = 0.4

Now we create an empty model and add the variables

In [ ]:
# Define a model
model2 = mip.Model()

# Define variables
x = {(str(i), j):model2.add_var(name=str(i)+','+j, lb=0) for i in I for j in J}
y = {j: model2.add_var(name=j, lb=0) for j in J}

Let us write the objective function: in the general case, it can be written as a sum over the set of models.

In [ ]:
# Define the objective function
model2.objective = mip.maximize(mip.xsum(y[j]*r[j] for j in J) - mip.xsum(c[i]*x[str(i), j] for i in I for j in J))

The constraints can be generated in loops:

In [ ]:
# CONSTRAINTS
# Availability constraint
for i in I:
  model2.add_constr(mip.xsum(x[str(i), j] for j in J) <= b[i])

# Conservation constraint
for j in J:
    model2.add_constr(mip.xsum(x[str(i), j] for i in I) == y[j])
# Maximum quantity
for i in I:
    for j in J:
        model2.add_constr(x[str(i), j] <= q_max[str(i),j]*y[j])
# Minimum quantity
for i in I:
    for j in J:
        model2.add_constr(x[str(i), j] >= q_min[str(i),j]*y[j])

The model is complete. Let's solve it and print the optimal solution.

In [ ]:
# Optimizing command
model2.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [ ]:
# Optimal objective function value
model2.objective.x

96000.0

In [ ]:
# Printing the variables values
for i in model2.vars:
    print(i.name, i.x)

0,A 500.0
0,B 4500.0
0,C 0.0
1,A 2400.0
1,B 0.0
1,C 0.0
2,A 0.0
2,B 4000.0
2,C 0.0
3,A 0.0
3,B 1500.0
3,C 0.0
A 2900.0
B 10000.0
C 0.0
